### Instalação Biblioteca

In [1]:
# Instalar bibliotecas ( necessário)
!pip install mlflow evidently scikit-learn pandas numpy matplotlib seaborn papermill

# Importar pacotes principais
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from evidently.report import Report
from evidently.metrics import DataDriftTable
import pickle



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Configurar MLflow para rastreamento de experimentos com objetivo de monitorar todo o processo, incluindo tempo de carregamento, transformações e qualidade dos dados. início do pipeline.

In [16]:
import mlflow
import os

# Definindo um novo diretório para armazenar os logs dos experimentos
caminho_logs = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Aplicando_para_dissertacao/mlflow_logs"

# Criar o diretório se não existir
os.makedirs(caminho_logs, exist_ok=True)

# Convertendo o caminho para o formato correto de URI
caminho_logs_uri = f"file:///{os.path.abspath(caminho_logs)}"

# Configurando o MLflow para usar esse diretório
mlflow.set_tracking_uri(caminho_logs_uri)

# Criando ou definindo o experimento
mlflow.set_experiment("Evasao_UFCG")

# Exibir a configuração para garantir que está correto
print(f"Tracking URI: {mlflow.get_tracking_uri()}")


Tracking URI: file:///E:\Mestrado UFCG\Semestre 2024.2\Dados\Aplicando_para_dissertacao\mlflow_logs


### Carregar os dados as tabelas relevantes

In [17]:
import pandas as pd
import os
import mlflow

# Definindo caminhos
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"
tables = ["alunos", "tabela_dados_ingresso", "tabela_dados_pessoais", "matriculas"]

# Iniciar Experimento MLflow
mlflow.set_experiment("Carregamento de Dados UFCG")

with mlflow.start_run():
    # Log do início do carregamento usando métrica em vez de parâmetro fixo
    mlflow.log_metric("status_iniciando", 1)

    dados = {}
    
    for table in tables:
        caminho = os.path.join(caminho_base, f"{table}.csv")
        try:
            # Detectar delimitador correto
            with open(caminho, "r", encoding="utf-8") as f:
                primeira_linha = f.readline()
                if ";" in primeira_linha:
                    delimitador = ";"
                elif "," in primeira_linha:
                    delimitador = ","
                elif "\t" in primeira_linha:
                    delimitador = "\t"
                else:
                    delimitador = ","  # Padrão caso não detecte

            # Carregar CSV com delimitador correto
            df = pd.read_csv(caminho, delimiter=delimitador, encoding="utf-8")
            
            # Verifica se carregou corretamente
            if df.shape[1] == 1:
                print(f" Atenção: {table} foi carregada com apenas 1 coluna! Pode haver erro no delimitador.")
            
            dados[table] = df
            print(f"{table} carregada com sucesso! ({df.shape[0]} linhas, {df.shape[1]} colunas)")
        except Exception as e:
            print(f"Erro ao carregar {table}: {e}")
            mlflow.log_param(f"erro_{table}", str(e))  # Log do erro no MLflow

    # Usar métrica ao invés de parâmetro para evitar erro no MLflow
    mlflow.log_metric("status_carregamento_finalizado", 1)

    print(" Carregamento finalizado.")


alunos carregada com sucesso! (3761 linhas, 22 colunas)
tabela_dados_ingresso carregada com sucesso! (3761 linhas, 4 colunas)
tabela_dados_pessoais carregada com sucesso! (3761 linhas, 5 colunas)
matriculas carregada com sucesso! (182660 linhas, 10 colunas)
 Carregamento finalizado.


### Visualizar colunas das tabelas

In [18]:
import pandas as pd

# Caminho base onde as tabelas estão armazenadas
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"

# Lista com os nomes das tabelas
tables = ["alunos", "tabela_dados_ingresso", "tabela_dados_pessoais", "matriculas"]

# Carregar e exibir as colunas de cada tabela
for tabela in tables:
    # Carregar a tabela especificando o separador ';'
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    
    # Mostrar as primeiras linhas e as colunas da tabela
    print(f"\nColunas da tabela '{tabela}':")
    print(tabela_df.columns.tolist())


Colunas da tabela 'alunos':
['MATRICULA', 'ID_CIDADAO', 'NOME', 'IDADE', 'E-MAIL', 'GENERO', 'ESTADO_CIVIL_ALUNOS', 'NACIONALIDADE', 'LOCAL_NASCIMENTO', 'ESTADO', 'TERMO_ESTADO', 'RAZAO_INATIVIDADE', 'TIPO_ADMISSAO', 'TERMO_ADMISSAO', 'POLITICA_AFIRMATIVA', 'TIPO_ENSINO_MEDIO', 'ANO_FORMATURA_ENSINO_MEDIO', 'CODIGO_CURSO', 'CODIGO_CURRICULAR', ' ALUNOS_ATIVOS', 'EX_ALUNOS', 'ALUNOS_INATIVOS']

Colunas da tabela 'tabela_dados_ingresso':
['MATRICULA', 'TIPO_ADMISSAO', 'TIPO_ENSINO_MEDIO', 'POLITICA_AFIRMATIVA_ALUNOS']

Colunas da tabela 'tabela_dados_pessoais':
['MATRICULA', 'IDADE', 'GENERO', 'LOCAL_NASCIMENTO', 'ESTADO']

Colunas da tabela 'matriculas':
['MATRICULA', 'CODIGO_DISCIPLINA', 'NOME', 'CREDITOS', 'HORAS', 'TERMO', 'ID_CLASS', 'NOTA', 'ESTATUS', 'TIPO']


### Filtro de Intervalo da pesquisa: 2001.1 - 2023.2

In [19]:
import pandas as pd

# Caminho base onde as tabelas estão armazenadas
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"

# Lista com os nomes das tabelas
tables = ["alunos", "tabela_dados_ingresso", "tabela_dados_pessoais", "matriculas"]

# Carregar e exibir as colunas de cada tabela
for tabela in tables:
    # Carregar a tabela especificando o separador ';'
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    
    # Mostrar as primeiras linhas e as colunas da tabela
    print(f"\nColunas da tabela '{tabela}':")
    print(tabela_df.columns.tolist())
    
    # Se a tabela for "matriculas", aplicar o filtro de ano e semestre
    if tabela == "matriculas":
        # Garantir que a coluna 'TERMO' esteja em formato adequado (se necessário)
        tabela_df['TERMO'] = tabela_df['TERMO'].astype(str)
        
        # Filtrar os registros entre 2002.1 e 2023.2
        tabela_df = tabela_df[tabela_df['TERMO'].between('2002.1', '2023.2')]
        
        # Exibir as primeiras linhas após o filtro
        print(f"\nPrimeiras linhas da tabela '{tabela}' após o filtro:")
        print(tabela_df.head())



Colunas da tabela 'alunos':
['MATRICULA', 'ID_CIDADAO', 'NOME', 'IDADE', 'E-MAIL', 'GENERO', 'ESTADO_CIVIL_ALUNOS', 'NACIONALIDADE', 'LOCAL_NASCIMENTO', 'ESTADO', 'TERMO_ESTADO', 'RAZAO_INATIVIDADE', 'TIPO_ADMISSAO', 'TERMO_ADMISSAO', 'POLITICA_AFIRMATIVA', 'TIPO_ENSINO_MEDIO', 'ANO_FORMATURA_ENSINO_MEDIO', 'CODIGO_CURSO', 'CODIGO_CURRICULAR', ' ALUNOS_ATIVOS', 'EX_ALUNOS', 'ALUNOS_INATIVOS']

Colunas da tabela 'tabela_dados_ingresso':
['MATRICULA', 'TIPO_ADMISSAO', 'TIPO_ENSINO_MEDIO', 'POLITICA_AFIRMATIVA_ALUNOS']

Colunas da tabela 'tabela_dados_pessoais':
['MATRICULA', 'IDADE', 'GENERO', 'LOCAL_NASCIMENTO', 'ESTADO']

Colunas da tabela 'matriculas':
['MATRICULA', 'CODIGO_DISCIPLINA', 'NOME', 'CREDITOS', 'HORAS', 'TERMO', 'ID_CLASS', 'NOTA', 'ESTATUS', 'TIPO']

Primeiras linhas da tabela 'matriculas' após o filtro:
       MATRICULA  CODIGO_DISCIPLINA                                     NOME  \
25990  194110044            1411170                        TEORIA DOS GRAFOS   
25991  19

In [20]:
import pandas as pd

# Caminho base onde as tabelas estão armazenadas
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"

# Lista com os nomes das tabelas
tables = ["alunos", "tabela_dados_ingresso", "tabela_dados_pessoais", "matriculas"]

# Carregar e exibir as colunas de cada tabela
for tabela in tables:
    # Carregar a tabela especificando o separador ';'
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    
    # Se a tabela for "matriculas", aplicar o filtro de ano e semestre
    if tabela == "matriculas":
        # Garantir que a coluna 'TERMO' esteja em formato adequado (se necessário)
        tabela_df['TERMO'] = tabela_df['TERMO'].astype(str)
        
        # Filtrar os registros entre 2002.1 e 2023.2
        tabela_df = tabela_df[tabela_df['TERMO'].between('2002.1', '2023.2')]
    
    # Mostrar as dimensões (linhas e colunas) da tabela
    print(f"\nTabela '{tabela}' tem {tabela_df.shape[0]} linhas e {tabela_df.shape[1]} colunas.")



Tabela 'alunos' tem 3761 linhas e 22 colunas.

Tabela 'tabela_dados_ingresso' tem 3761 linhas e 4 colunas.

Tabela 'tabela_dados_pessoais' tem 3761 linhas e 5 colunas.

Tabela 'matriculas' tem 136066 linhas e 10 colunas.


### Remover dados duplicados ou não relacionados

In [21]:
import pandas as pd

# Caminho base onde as tabelas estão armazenadas
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"

# Lista com os nomes das tabelas
tables = ["alunos", "tabela_dados_ingresso", "tabela_dados_pessoais", "matriculas"]

# Carregar e exibir as colunas de cada tabela
for tabela in tables:
    # Carregar a tabela especificando o separador ';'
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    
    # Se a tabela for "matriculas", aplicar o filtro de ano e semestre
    if tabela == "matriculas":
        # Garantir que a coluna 'TERMO' esteja em formato adequado (se necessário)
        tabela_df['TERMO'] = tabela_df['TERMO'].astype(str)
        
        # Filtrar os registros entre 2002.1 e 2023.2
        tabela_df = tabela_df[tabela_df['TERMO'].between('2002.1', '2023.2')]
        
        # Remover duplicatas com base na coluna 'MATRICULA' (ou qualquer outra que identifique um aluno único)
        tabela_df = tabela_df.drop_duplicates(subset=['MATRICULA'], keep='first')  # Mantém a primeira ocorrência de cada matrícula
    
    # Mostrar as dimensões (linhas e colunas) da tabela após o tratamento
    print(f"\nTabela '{tabela}' tem {tabela_df.shape[0]} linhas e {tabela_df.shape[1]} colunas após tratamento de duplicação.")



Tabela 'alunos' tem 3761 linhas e 22 colunas após tratamento de duplicação.

Tabela 'tabela_dados_ingresso' tem 3761 linhas e 4 colunas após tratamento de duplicação.

Tabela 'tabela_dados_pessoais' tem 3761 linhas e 5 colunas após tratamento de duplicação.

Tabela 'matriculas' tem 3789 linhas e 10 colunas após tratamento de duplicação.


### A Identificar e Corrigir formatação

In [22]:
import pandas as pd

# Caminho base onde as tabelas estão armazenadas
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"

# Lista com os nomes das tabelas
tables = ["alunos", "tabela_dados_ingresso", "tabela_dados_pessoais", "matriculas"]

# Função para corrigir formatação
def corrigir_formatacao(df):
    # Corrigir espaços em branco nas colunas de texto
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
    # Converter colunas numéricas com erro de formatação para o tipo numérico
    for col in df.select_dtypes(include=['object']).columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Converte para numérico, coloca NaN em erros
        except ValueError:
            pass  # Caso a conversão falhe, ignora
    return df

# Carregar e corrigir as tabelas
for tabela in tables:
    # Carregar a tabela especificando o separador ';'
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    
    # Corrigir formatação
    tabela_df = corrigir_formatacao(tabela_df)
    
    # Mostrar as dimensões (linhas e colunas) da tabela após a correção de formatação
    print(f"\nTabela '{tabela}' tem {tabela_df.shape[0]} linhas e {tabela_df.shape[1]} colunas após correção de formatação.")
    
    # Exibir as primeiras linhas para revisão
    print(tabela_df.head())


C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)



Tabela 'alunos' tem 3761 linhas e 22 colunas após correção de formatação.
   MATRICULA    ID_CIDADAO  NOME  IDADE  E-MAIL  GENERO  ESTADO_CIVIL_ALUNOS  \
0  102210001           NaN   NaN     42     NaN     NaN                  NaN   
1  102210002  5.175670e+09   NaN     43     NaN     NaN                  NaN   
2  102210003           NaN   NaN     40     NaN     NaN                  NaN   
3  102210004  2.454245e+08   NaN     43     NaN     NaN                  NaN   
4  102210005  1.195394e+09   NaN     43     NaN     NaN                  NaN   

   NACIONALIDADE  LOCAL_NASCIMENTO  ESTADO  ...  TIPO_ADMISSAO  \
0            NaN               NaN     NaN  ...            NaN   
1            NaN               NaN     NaN  ...            NaN   
2            NaN               NaN     NaN  ...            NaN   
3            NaN               NaN     NaN  ...            NaN   
4            NaN               NaN     NaN  ...            NaN   

   TERMO_ADMISSAO  POLITICA_AFIRMATIVA  TIPO_EN

C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)



Tabela 'matriculas' tem 182660 linhas e 10 colunas após correção de formatação.
   MATRICULA  CODIGO_DISCIPLINA  NOME  CREDITOS  HORAS   TERMO  ID_CLASS  \
0  177110007            1307045   NaN         5     75  1977.1       NaN   
1  177110007            1109002   NaN         4     60  1977.2       NaN   
2  177110007            1304009   NaN         2     30  1977.2       NaN   
3  177110007            1307012   NaN         3     45  1977.2       NaN   
4  177110007            1109007   NaN         4     60  1978.1       NaN   

   NOTA  ESTATUS  TIPO  
0   6.1      NaN   NaN  
1   5.2      NaN   NaN  
2   6.0      NaN   NaN  
3   8.0      NaN   NaN  
4   5.1      NaN   NaN  


### Vamos visualizar colunas valores ausentes 

In [33]:
# Dicionário para armazenar as tabelas carregadas
tabelas_carregadas = {}

# Carregar e corrigir as tabelas
for tabela in tables:
    # Carregar a tabela especificando o separador ';'
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    
    # Corrigir formatação
    tabela_df = corrigir_formatacao(tabela_df)
    
    # Armazenar a tabela no dicionário
    tabelas_carregadas[tabela] = tabela_df

# Verificar colunas com valores ausentes em cada tabela
for tabela in tables:
    tabela_df = tabelas_carregadas[tabela]  # Acessa a tabela carregada do dicionário
    
    # Verificar quais colunas contêm valores ausentes
    colunas_ausentes = tabela_df.columns[tabela_df.isnull().any()]
    
    if len(colunas_ausentes) > 0:
        print(f"\nTabela '{tabela}' - Colunas com valores ausentes:")
        print(colunas_ausentes.tolist())
    else:
        print(f"\nTabela '{tabela}' não contém colunas com valores ausentes.")


C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Big Data\AppData\Local\Temp\ipykernel_2832\4044100935.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)



Tabela 'alunos' - Colunas com valores ausentes:
['ID_CIDADAO', 'NOME', 'E-MAIL', 'GENERO', 'ESTADO_CIVIL_ALUNOS', 'NACIONALIDADE', 'LOCAL_NASCIMENTO', 'ESTADO', 'TERMO_ESTADO', 'RAZAO_INATIVIDADE', 'TIPO_ADMISSAO', 'POLITICA_AFIRMATIVA', 'TIPO_ENSINO_MEDIO', 'ANO_FORMATURA_ENSINO_MEDIO']

Tabela 'tabela_dados_ingresso' - Colunas com valores ausentes:
['TIPO_ADMISSAO', 'TIPO_ENSINO_MEDIO', 'POLITICA_AFIRMATIVA_ALUNOS']

Tabela 'tabela_dados_pessoais' - Colunas com valores ausentes:
['GENERO', 'LOCAL_NASCIMENTO', 'ESTADO']

Tabela 'matriculas' - Colunas com valores ausentes:
['NOME', 'ID_CLASS', 'ESTATUS', 'TIPO']


### Modificar e remover campos de dados incorretos ou incompletos Remover o excesso de amostra Padronizar os dados limpos

### Realizar o merges das Tabelas

In [15]:
# Agora, realizar o merge das tabelas
# Vamos começar com a tabela "matriculas" como base
df_completo = pd.read_csv(caminho_base + tables[4] + ".csv", sep=';', on_bad_lines='skip')

# Realizar o merge das outras tabelas
for tabela in tables[:4]:  # As 4 primeiras tabelas na lista
    tabela_df = pd.read_csv(caminho_base + tabela + ".csv", sep=';', on_bad_lines='skip')
    if 'MATRICULA' in tabela_df.columns:
        df_completo = df_completo.merge(tabela_df, on='MATRICULA', how='left')

# Verificar as primeiras linhas do dataframe final
print("\nDataFrame final após merge:")
print(df_completo.head())


DataFrame final após merge:
   MATRICULA  CODIGO_DISCIPLINA                    NOME_x  CREDITOS  HORAS  \
0  177110007            1307045         LINGUA PORTUGUESA         5     75   
1  177110007            1109002      INTRODU??O ? ?LGEBRA         4     60   
2  177110007            1304009           EDUCA??O F?SICA         2     30   
3  177110007            1307012                  INGLES I         3     45   
4  177110007            1109007  CALCULO DIF E INTEGRAL I         4     60   

    TERMO  ID_CLASS  NOTA   ESTATUS    TIPO  ...  EX_ALUNOS ALUNOS_INATIVOS  \
0  1977.1       NaN   6.1  APROVADO  NORMAL  ...        NaN             NaN   
1  1977.2       NaN   5.2  APROVADO  NORMAL  ...        NaN             NaN   
2  1977.2       NaN   6.0  APROVADO  NORMAL  ...        NaN             NaN   
3  1977.2       NaN   8.0  APROVADO  NORMAL  ...        NaN             NaN   
4  1978.1       NaN   5.1  APROVADO  NORMAL  ...        NaN             NaN   

   RAZAO_DE_INATIVIDADE_DE_